In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests


In [25]:
dataset = pd.read_csv('malicious_phish1.csv')
dataset.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [26]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [27]:
dataset["type"].value_counts()

type
benign        428103
defacement     96457
phishing       94111
malware        32520
Name: count, dtype: int64

In [28]:
# filter the dataset and keep the urls starts with http or https
dataset = dataset[(dataset.url.str.startswith('http://')) | (dataset.url.str.startswith('https://'))]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187997 entries, 3 to 555185
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     187997 non-null  object
 1   type    187997 non-null  object
dtypes: object(2)
memory usage: 4.3+ MB


In [29]:
dataset["type"].value_counts()

type
defacement    96457
benign        35378
malware       31310
phishing      24852
Name: count, dtype: int64

In [ ]:
# balance the dataset by removing the extra rows of benign urls